<a href="https://colab.research.google.com/github/ratral/colab_control_valves/blob/master/cValvesCurator_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1.- Installing an R Packages and functions.

_Read libraries_

In [ ]:
requiredPackages <- c("tidyverse", "here", "devtools")

  for(p in requiredPackages){
    if(!require( p, character.only = TRUE)) install.packages(p)
    library( p, character.only = TRUE)
  }

_Read hydraulic and valves functions_

In [ ]:

  devtools::install_github( "ratral/wcontrolvalve")

# 2.- The issue to be solved (Control Valve Selection)

The objective of this exercise is to select between three possible types of control valves (the same valve but different types of cylinders) the most suitable for the operation of the system.

The conditions are as follows:

  - the diameter of the valve is smaller than the diameter of the pipes;
  -  the system requires a constant flow,
  - the pressures upstream and downstream of the valve vary during the operation of the system;
  - Initially, only three operational points will be evaluated.

The steps to follow are those:

   - Step 1. Specify the data and variables required to size the valve.


## Step 1. Specify the data and variables required to size the valve.

### Basic data

In [3]:
  temp <- 15.00   # Upstream water temperature in centigrade (°C)
  elev <- 2700    # Elevation above sea level - in meter - 
  flow <- 9396    # Flow in m3/h
  d    <- 1.000   # Diameter of the valve in meter
  d1   <- 1.4     # Diameter of the Pipe Upstream in meter
  d2   <- 1.2     # Diameter of the Pipe Downstream in meter

### Minimum, mean, and maximum pressures upstream and downstream of the valve and flow. 

(Pressure in $bar$ and flow in $m^3/hr$)

In [4]:
  base_data <- tribble(
    ~measurement, ~p1,   ~p2, ~flow,
    "Min.",      1.720, 0.614,  9396,
    "Mean",      1.410, 0.684,  9396,
    "Max.",      1.370, 1.009,  9396
  ) 

### Flow characteristic curve of the valve (inherent flow characteristic of the valve).

  - parameters of the flow characteristic curve:
    - $b$ denoting the steepness of the low characteristic curve, 
    - $d$ is the upper asymptotes or limits of the response, and, 
    - $e$ is the effective dose. 
  - Loss Coefficient $(\zeta_{v_s})$ is the Zeta value of the valve by 100% opening.
  - $F_{L_s}$ is the Liquid Pressure Recovery Factor by 100%

In [5]:
  cylinder <- tribble(
    ~typ,         ~b,    ~d,      ~e,   ~zvs,  ~fls,
    "typ_01", -2.393,	1.389,  67.422,	 1.780,	0.617, # 0
    "typ_02", -3.893,	1.146,  61.038,  3.000,	0.598, # 20-30
    "typ_03", -2.755, 1.979,  99.215, 15.830, 0.775  # 40
  ) 
  

## Step 2. Calculation $ K_{v_s}$, $F_{P_s}$ and $F_{LP_s}$ 

- Flow coefficient by 100% opening : $ K_{v_s} = \frac{d^2}{\sqrt{626.3}} \cdot \frac{1}{\sqrt{\zeta_{v_s}}}$

- Piping geometry factor by 100% opening : $F_{P_s} = \frac{1}{\sqrt{1+\frac{\sum{\zeta} \cdot \left(\frac{K_{v_s}}{d^2} \right)^2}{0.0016}}}$

- Combined liquid pressure recovery factor: $F_{LP_s} = \frac{F_{L_s}}{\sqrt{1 + F_{L_s} \cdot \frac{\sum{\zeta} \cdot \left(\frac{K_{v_s}}{d^2} \right)^2}{0.0016}}}$ 

In this equations, the factor $\sum \zeta$ is the algebraic sum of all effective resistance coefficients of all fittings that are attached to the control valve. This does not include the drag coefficient of the control valve itself.

$ \sum{\zeta} = (\zeta_{1} + \zeta_{2}) + (\zeta_{B_1} - \zeta_{B_2})$


Where:

  - $\zeta_1$ is the resistance coefficient of the valve in the inlet : $\zeta_1 = 0.5 \cdot \left( 1 - \left( \frac{d}{D_1}\right)^2 \right)^2$
  - $\zeta_{2}$ is the resistance coefficient of the valve in the outlet : $\zeta_2 = \left( 1 - \left( \frac{d}{D_2}\right)^2 \right)^2$
  - $\zeta_{B_1}$  is the Bernoulli pressure number in the valve inlet : $\zeta_{B_1} = 1- \left( \frac{d}{D_1}  \right)^4$
  - $\zeta_{B_2}$  is the Bernoulli pressure number in the valve outlet :$\zeta_{B_2} = 1- \left( \frac{d}{D_2}  \right)^4$







In [9]:
  cylinder <- cylinder %>% 
    mutate( kvs  = wcontrolvalve::kv_value(d, zvs)) %>% 
    mutate( fps  = wcontrolvalve::fp(kvs, d*1000, d1*1000, d2*1000)) %>% 
    mutate( flps = wcontrolvalve::flp(kvs, fls, d*1000, d1*1000, d2*1000)) %>% 
    mutate( flps_fps = flps/fps) %>% 
    select( typ, b, d, e, kvs, zvs,  fls, fps, flps, flps_fps)

In [ ]:
# Table